In [14]:
import pandas as pd
import numpy as np 

data_tic_tak  = pd.read_csv("./tic-tac-toe.csv")

In [15]:
#print(data.head())
attributes = data_tic_tak.columns.values
print(attributes)
attribute_values = data_tic_tak.iloc[:,0:9]
#print(attribute_values)
class_labels  = data_tic_tak.iloc[:,9]
#print(class_labels)

#decisionTree = DecisionTree(attribute_values, attributes, class_labels)
print(len(class_labels))

['top-left-square' 'top-middle-square' 'top-right-square'
 'middle-left-square' 'middle-middle-square' 'middle-right-square'
 'bottom-left-square' 'bottom-middle-square' 'bottom-right-square' 'Class']
958


In [17]:
import numpy as np
import math 
import operator

def CalEntropy(data, column_name):
    '''
    '''
    #print("***********", column_name)
    total_data_samples = len(data)
    #print("Total data samples", total_data_samples)

    col_feature_values = data[column_name]
    (vals, vals_count) = np.unique(col_feature_values, return_counts=True)
    #print("Values in the colum and there count", vals, vals_count)

    label_values = data.iloc[:,-1]
    class_labels  = np.unique(label_values)
    #print("Class Labels", class_labels)

    data_temp =  pd.concat([col_feature_values, label_values], axis=1)
    #print(data_temp.head())

    #class count for each feature variable
    #dictionary structure {'attribute value': [negetive sample, positive_sample]}
    label_count_dict = {}
    #print(data_temp.head())
    for x in vals:
        #print(column_name)
        d2 = data_temp.loc[data_temp[column_name] == x]
        d2 = d2.iloc[:,-1]
        (class_vals, class_vals_count) = np.unique(d2, return_counts=True)
        #print(class_vals, class_vals_count)  
        label_count_dict[x] = class_vals_count
    #print("$$$$$$$$ Label count dictionary is", label_count_dict)

    #entropy list stores the enropy for each feature variable 
    entropy_list = []
    #calculate entropy as -plog(p)
    for key in label_count_dict:
        if(len(label_count_dict[key]) > 1):
            neg_count = label_count_dict[key][0]
            pos_count = label_count_dict[key][1]
        #if ((pos_count != 0) or (neg_count != 0)):
            p1 = pos_count/(pos_count + neg_count)
            p2 = neg_count/(pos_count + neg_count)
            entropy = -((p1*math.log2(p1))+ (p2*math.log2(p2)))
            #print("Value of entropy is", entropy)
            entropy_list.append(entropy)
        else:
            entropy_list.append(0)
    #print("Entropy list", entropy_list)

    #Calculating the total entropy for the column 
    total_entropy = 0
    for x  in range(0, len(entropy_list)):
        total_entropy = total_entropy + (entropy_list[x]*(vals_count[x]/total_data_samples))
    #print("Total Entropy", total_entropy)

    return total_entropy

def CalInformationGain(data, attributes):
    '''
    '''
    total_data_samples = len(data)
    #print("Total data samples", total_data_samples)  
    label_values = data.iloc[:,-1]
    
    class_labels, label_counts  = np.unique(label_values, return_counts=True)
    #print("Class Labels", class_labels)
    #print("Label Count", label_counts)

    raw_entropy = 0
    for x  in range(0, len(label_counts)):
        p = label_counts[x]/total_data_samples
        raw_entropy = raw_entropy -(p*math.log2(p))
    #print("Raw entropy", raw_entropy)

    #information_gain_dict dictionary stores label of colum and IG corresponding to that 
    information_gain_dict = {}
    #attributes = data.iloc[:,0:9].columns.values
    for x in attributes:
        information_gain_dict[x] = raw_entropy - CalEntropy(data, x)

    #print("Dict of IG", (information_gain_dict))
    return information_gain_dict

def select_best_feature(information_gain_dict):
    sorted_information_gain_dict = sorted(information_gain_dict.items(), key=operator.itemgetter(1), reverse=True)
    #print(type(sorted_information_gain_dict))
    max_ig_feature = next(iter(sorted_information_gain_dict))
    ret = max_ig_feature[0]
    print(ret)
    return ret 

class Node:
    def __init__(self, value=None, edge=None):
        self.value = value
        self.edge = edge #edge to this node (coming to this node)
        self.children = [] #list of Nodes
    def __str__(self):
        st = 'Node.value = '+str(self.value)+' edge_coming_to_me = '+str(self.edge)
        return st

#id3 tree will return the root node 
def id3(data, attributes, node):
    print("Attributes for this iteration are", attributes)
    root = Node()
    root.edge = node.edge

    #check if all examples are positive or negative 
    class_labels = data.iloc[:,-1]
    labels, label_count = np.unique(class_labels, return_counts=True)
    #print(labels, label_count)

    if (len(labels) == 1):
        root.value = labels
        return root

    '''
    if (len(data) == 0):
        root.value = "END"
        return root
    '''
    '''
    for x in range(0,len(labels)):
        if label_count[x] == 0:
            root.value = labels[(x+1)%2]
            return root
    '''

    if attributes.size == 0:
        if(label_count[0]>=label_count[1]):
            root.value = labels[0]
        else:
            root.value = labels[1]
        return root
    else:
        information_gain_dict = CalInformationGain(data, attributes)
        A = select_best_feature(information_gain_dict)
        root.value = A
        root.children = []
        values_of_A = data[A]
        for e in values_of_A.unique():
            child_node = Node()
            #child_node.value = e
            child_node.edge = e
            #root.children.append(child_node)
            #dominant_label = data['Class'].value_counts()[0]
            data2 = data.loc[data[A] == e]
            #if len(data) == 0:
            #    child_node.edge = dominant_label
            if(len(data)>0):
                new_attributes = attributes[attributes != A]
                #TODO:
            #child_node.edge = id3(data, new_attributes)
            child_node = id3(data2, new_attributes, child_node)
            root.children.append(child_node)

    return root

#information_gain_dict = CalInformationGain(data, attributes)
#select_best_feature(information_gain_dict)
attributes = data_tic_tak.iloc[:,0:9].columns.values
#print(attributes)
#X = data.drop('Class', axis=1)
#y = data['Class']

#data1 = pd.read_csv("./playtennis.csv")
#attributes1 = data1.iloc[:,1:5].columns.values

temp_node = Node()
root = id3(data_tic_tak, attributes, temp_node)

iddle-square' 'middle-left-square' 'middle-right-square'
 'bottom-left-square' 'bottom-middle-square']
top-middle-square
Attributes for this iteration are ['middle-left-square' 'middle-right-square' 'bottom-left-square'
 'bottom-middle-square']
middle-right-square
Attributes for this iteration are ['middle-left-square' 'bottom-left-square' 'bottom-middle-square']
Attributes for this iteration are ['middle-left-square' 'bottom-left-square' 'bottom-middle-square']
Attributes for this iteration are ['middle-left-square' 'bottom-left-square' 'bottom-middle-square']
Attributes for this iteration are ['middle-left-square' 'middle-right-square' 'bottom-left-square'
 'bottom-middle-square']
Attributes for this iteration are ['middle-left-square' 'middle-right-square' 'bottom-left-square'
 'bottom-middle-square']
Attributes for this iteration are ['top-middle-square' 'middle-left-square' 'middle-right-square'
 'bottom-left-square' 'bottom-middle-square']
bottom-left-square
Attributes for this i

In [19]:
def printNode(node):
    #print(node.edge)
    #print(node.value)
    st = 'Node.value = '+str(node.value)+' edge_coming_to_me = '+str(node.edge)
    print(st)

def printTree(node):
    if node:
        printNode(node)
        for c in node.children:
           printTree(c)
    else:
        return

printTree(root)

Node.value = middle-middle-square edge_coming_to_me = None
Node.value = top-left-square edge_coming_to_me = o
Node.value = top-right-square edge_coming_to_me = x
Node.value = top-middle-square edge_coming_to_me = x
Node.value = ['positive'] edge_coming_to_me = x
Node.value = bottom-middle-square edge_coming_to_me = o
Node.value = bottom-left-square edge_coming_to_me = x
Node.value = middle-left-square edge_coming_to_me = x
Node.value = ['positive'] edge_coming_to_me = x
Node.value = ['negative'] edge_coming_to_me = o
Node.value = middle-left-square edge_coming_to_me = o
Node.value = ['positive'] edge_coming_to_me = o
Node.value = ['negative'] edge_coming_to_me = x
Node.value = ['negative'] edge_coming_to_me = b
Node.value = middle-left-square edge_coming_to_me = b
Node.value = ['positive'] edge_coming_to_me = x
Node.value = middle-right-square edge_coming_to_me = o
Node.value = ['positive'] edge_coming_to_me = x
Node.value = ['negative'] edge_coming_to_me = o
Node.value = ['positive'] 